Extracing the cofee prices 

In [5]:
import blpapi
import sys
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

def get_coffee_daily_data():
    """
    Retrieves the last 5 years of daily price data for coffee commodity ("KC1 Comdty")
    with fields: PX_OPEN, PX_HIGH, PX_LOW, PX_LAST, and PX_VOLUME.
    
    Returns:
        pandas.DataFrame: Dataframe indexed by date containing the retrieved fields.
    """
    # Set up the Bloomberg API session options.
    sessionOptions = blpapi.SessionOptions()
    sessionOptions.setServerHost("localhost")
    sessionOptions.setServerPort(8194)
    
    # Initialize and start the session.
    session = blpapi.Session(sessionOptions)
    if not session.start():
        print("Failed to start session.")
        sys.exit(1)
    
    # Open the reference data service.
    if not session.openService("//blp/refdata"):
        print("Failed to open //blp/refdata")
        sys.exit(1)
        
    refDataService = session.getService("//blp/refdata")
    
    # Create a HistoricalDataRequest.
    request = refDataService.createRequest("HistoricalDataRequest")
    
    # Define the security and fields for the request.
    request.getElement("securities").appendValue("KC1 Comdty")
    fields = ["PX_OPEN", "PX_HIGH", "PX_LOW", "PX_LAST", "PX_VOLUME"]
    for field in fields:
        request.getElement("fields").appendValue(field)
    
    # Set the date range: last 5 years until today.
    today = date.today()
    start_date = today - relativedelta(years=5)
    request.set("startDate", start_date.strftime("%Y%m%d"))
    request.set("endDate", today.strftime("%Y%m%d"))
    
    # Set periodicity to DAILY.
    request.set("periodicitySelection", "DAILY")
    
    # Send the request.
    session.sendRequest(request)
    
    # Process the response and store data.
    data = []
    while True:
        event = session.nextEvent(500)
        for msg in event:
            if msg.hasElement("securityData"):
                securityData = msg.getElement("securityData")
                if securityData.hasElement("fieldData"):
                    fieldData = securityData.getElement("fieldData")
                    for i in range(fieldData.numValues()):
                        dayData = fieldData.getValueAsElement(i)
                        # Build a row with the date and requested fields.
                        row = {"date": dayData.getElementAsString("date")}
                        for field in fields:
                            # Use .hasElement() to safely check for the field.
                            row[field] = dayData.getElementAsFloat(field) if dayData.hasElement(field) else None
                        data.append(row)
        # Break when the response event is complete.
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    # Stop the session.
    session.stop()
    
    # Convert the retrieved data into a pandas DataFrame.
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)
    
    return df

# Example usage:
if __name__ == "__main__":
    df_coffee = get_coffee_daily_data()
    print(df_coffee)
    df_coffee.to_csv("Coffee.csv")


            PX_OPEN  PX_HIGH  PX_LOW  PX_LAST  PX_VOLUME
date                                                    
2020-04-13   118.95   120.60  116.60   119.75    14231.0
2020-04-14   119.90   120.35  115.75   117.20    14615.0
2020-04-15   117.00   121.20  114.60   120.20     9063.0
2020-04-16   119.85   120.85  117.90   118.60     7041.0
2020-04-17   118.30   119.60  115.70   116.05    10691.0
...             ...      ...     ...      ...        ...
2025-04-07   362.50   364.50  338.60   344.80    37079.0
2025-04-08   345.00   350.15  340.50   342.90    25850.0
2025-04-09   341.15   353.75  325.20   341.70    28737.0
2025-04-10   360.10   361.50  342.15   342.85    20192.0
2025-04-11   345.95   360.70  341.60   360.30    22044.0

[1259 rows x 5 columns]


In [3]:
import blpapi
import sys
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

def get_and_save_historical_data(ticker, frequency="DAILY", fields=None):
    """
    Retrieves the last 5 years of historical Bloomberg data for a given ticker,
    saves the data to a CSV named after the ticker, and returns the dataframe.
    
    Args:
        ticker (str): Bloomberg ticker (e.g., "CSCITLTL Index").
        frequency (str): Data frequency, for example "DAILY" or "MONTHLY".
                         (Default is "DAILY".)
        fields (list of str): Bloomberg fields to retrieve. Defaults to:
            ["PX_OPEN", "PX_HIGH", "PX_LOW", "PX_LAST", "PX_VOLUME"]
    
    Returns:
        pd.DataFrame: Historical data with dates as the index.
    """
    # Use default fields if none are provided.
    if fields is None:
        fields = ["PX_OPEN", "PX_HIGH", "PX_LOW", "PX_LAST", "PX_VOLUME"]
    
    # Set up Bloomberg API session options.
    sessionOptions = blpapi.SessionOptions()
    sessionOptions.setServerHost("localhost")
    sessionOptions.setServerPort(8194)
    
    # Start the session.
    session = blpapi.Session(sessionOptions)
    if not session.start():
        print("Failed to start session.")
        sys.exit(1)
    
    # Open the reference data service.
    if not session.openService("//blp/refdata"):
        print("Failed to open //blp/refdata")
        sys.exit(1)
    
    refDataService = session.getService("//blp/refdata")
    # Create the HistoricalDataRequest.
    request = refDataService.createRequest("HistoricalDataRequest")
    
    # Append the requested ticker.
    request.getElement("securities").appendValue(ticker)
    
    # Append each field to the request.
    for field in fields:
        request.getElement("fields").appendValue(field)
    
    # Define the date range as the last 5 years.
    today = date.today()
    start_date = today - relativedelta(years=5)
    request.set("startDate", start_date.strftime("%Y%m%d"))
    request.set("endDate", today.strftime("%Y%m%d"))
    
    # Set the periodicity selection.
    request.set("periodicitySelection", frequency.upper())
    
    # Send the request.
    session.sendRequest(request)
    
    # Process the response.
    data = []
    while True:
        event = session.nextEvent(500)
        for msg in event:
            if msg.hasElement("securityData"):
                securityData = msg.getElement("securityData")
                if securityData.hasElement("fieldData"):
                    fieldData = securityData.getElement("fieldData")
                    for i in range(fieldData.numValues()):
                        dayData = fieldData.getValueAsElement(i)
                        # Build a dictionary for each row, starting with the date.
                        row = {"date": dayData.getElementAsString("date")}
                        for field in fields:
                            # Check if the field is available for that day.
                            if dayData.hasElement(field):
                                row[field] = dayData.getElementAsFloat(field)
                            else:
                                row[field] = None
                        data.append(row)
        # When the complete RESPONSE event is received, exit loop.
        if event.eventType() == blpapi.Event.RESPONSE:
            break
    
    # Stop the Bloomberg session.
    session.stop()
    
    # Convert list of dictionaries to a pandas DataFrame.
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)
    
    # Generate a filename for the CSV by replacing spaces with underscores.
    filename = f"{ticker.replace(' ', '_')}.csv"
    df.to_csv(filename)
    print(f"Data for {ticker} saved to '{filename}'.")
    
    return df

# Example usage: Retrieve and save data for multiple tickers.
if __name__ == "__main__":
    # Define a list of ticker info with desired frequency.
    tickers_info = [
        {"ticker": "CSCITLTL Index", "frequency": "DAILY"},   # ICE Total Exchange Total Coffee Inventories Data
        {"ticker": "CECFTOTV Index", "frequency": "MONTHLY"},   # Cecafe Brazil Coffee Monthly Volume Exports
        {"ticker": "CSCIBZTL Index", "frequency": "DAILY"},     # ICE Brazil Origin Coffee Inventory Data/Total
        {"ticker": "CECDARD Index",  "frequency": "DAILY"},     # Brazil Cecafe Data - Arabica Daily Exports
        {"ticker": "BZCTCTTT Index", "frequency": "MONTHLY"},   # Brazil CONAB Coffee Total Trees Monthly Estimate
        {"ticker": "BREXKD Index",   "frequency": "MONTHLY"},   # Brazil Coffee Exports to Country Value and Volume Monthly
        {"ticker": "CECDTOTD Index", "frequency": "DAILY"},     # Brazil Cecafe Data - Total Daily Exports
        {"ticker": "CCEIIQTL Index", "frequency": "MONTHLY"}    # China Customs Total Coffee Import Quantity
    ]
    
    # Loop through each ticker, retrieve its data, and save as CSV.
    for info in tickers_info:
        print(f"\nFetching data for {info['ticker']} ({info['frequency']} data)...")
        df = get_and_save_historical_data(info["ticker"], info["frequency"])
        print(df.head())



Fetching data for CSCITLTL Index (DAILY data)...
Data for CSCITLTL Index saved to 'CSCITLTL_Index.csv'.
           PX_OPEN PX_HIGH PX_LOW    PX_LAST PX_VOLUME
date                                                  
2020-04-13    None    None   None  1903323.0      None
2020-04-14    None    None   None  1901183.0      None
2020-04-15    None    None   None  1866211.0      None
2020-04-16    None    None   None  1866211.0      None
2020-04-17    None    None   None  1878805.0      None

Fetching data for CECFTOTV Index (MONTHLY data)...
Data for CECFTOTV Index saved to 'CECFTOTV_Index.csv'.
           PX_OPEN PX_HIGH PX_LOW    PX_LAST PX_VOLUME
date                                                  
2020-04-30    None    None   None  3642656.0      None
2020-05-31    None    None   None  3281713.0      None
2020-06-30    None    None   None  3075617.0      None
2020-07-31    None    None   None  3242294.0      None
2020-08-31    None    None   None  3573958.0      None

Fetching data for